# 문제 2-2 : 영화 리뷰 감정 분석기
### 출력 파서: EnumOutputParser

### 문제
#### 영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.

### 요구사항:
#### EnumOutputParser와 Enum 클래스 사용
#### 여러 개의 테스트 리뷰로 검증
#### 감정 분류 결과를 깔끔하게 출력
#### 테스트 리뷰 예시:
#### "이 영화 정말 재미없어요. 시간 낭비였습니다."
#### "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
#### "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."

#### 분석실패했을 경우에 사용하는 
#### OutputFixingParser 와 EnumOutputParser 를 같이 사용하는 것은 선택입니다.

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_t


In [3]:
from enum import Enum
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 보통


In [4]:
# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요.
2. 왜 그 단어로 답변했는지 간단한 설명도 같이 출력하세요.

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 보통'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 텍스트 감정 분석 전문가입니다.\n다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요.\n2. 왜 그 단어로 답변했는지 간단한 설명도 같이 출력하세요.\n\n답변:'), additional_kwargs={})]


In [5]:
# ChatOpenAI 모델 초기화
# 환경변수에서 OpenAI API 키 설정 (실제 사용시 주석 해제)
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# model = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=0  # 일관성을 위해 0으로 설정
# )
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
print(fixing_parser)

print("모델 및 파서 설정 완료")

parser=EnumOutputParser(enum=<enum 'Sentiment'>) retry_chain=PromptTemplate(input_variables=['completion', 'error', 'instructions'], input_types={}, partial_variables={}, template='Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019D0CDDE9F0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019D0D1326C0>, root_client=<openai.OpenAI object at 0x0000019D0CE133E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000019D0CEAC6B0>, model_name='moonshotai/kimi-k2-instruct-0905', temperature=0.0, model_kwargs={}, openai_api_key=Secre

In [ ]:
# 테스트 텍스트
texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",          # 부정
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",        # 긍정
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",        # 보통
    "OST는 좋았지만 전개가 늘어졌고 결말이 약했어요.",           # 혼합(보통 또는 부정)
    "연출이 신선했고 영상미가 뛰어났습니다. 다시 보고 싶어요."    # 긍정
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

테스트할 텍스트 5개 준비 완료


In [7]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        # use_fixing_parser가 True이면 OutputFixingParser가 적용되고, False이면 EnumOutputParser가 적용됨
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 

4. 텍스트: OST는 좋았지만 전개가 늘어졌고 결말이 약했어요.
   감정: 부정 

5. 텍스트: 연출이 신선했고 영상미가 뛰어났습니다. 다시 보고 싶어요.
   감정: 긍정 

=== 결과 요약 ===
성공: 5/5 (100.0%)
실패: 0/5


In [10]:
# 추가 기능: 배치 처리 및 통계
def batch_sentiment_analysis(text_list):
    """여러 텍스트를 한 번에 처리하고 통계 제공"""
    results = {
        '긍정': 0,
        '부정': 0,
        '중립': 0,
        '오류': 0
    }
    
    detailed_results = []
    
    for text in text_list:
        result, error = safe_sentiment_analysis(text)
        
        if result:
            sentiment = result.value
            results[sentiment] += 1
            detailed_results.append((text, sentiment, None))
        else:
            results['오류'] += 1
            detailed_results.append((text, None, error))
    
    return results, detailed_results

# 통계 출력 함수
def print_statistics(results, detailed_results):
    """결과 통계 출력"""
    print("\n=== 감정 분석 통계 ===")
    total = sum(results.values())
    
    for sentiment, count in results.items():
        percentage = (count / total * 100) if total > 0 else 0
        print(f"{sentiment}: {count}개 ({percentage:.1f}%)")
    
    print("\n=== 상세 결과 ===")
    for i, (text, sentiment, error) in enumerate(detailed_results, 1):
        status = sentiment if sentiment else " 오류"
        print(f"{i}. {text[:30]}... → {status}")

# 배치 처리 시뮬레이션
print("\n=== 배치 처리 시뮬레이션 ===")
simulated_results = {
    '긍정': 2,
    '부정': 2, 
    '중립': 1
}

simulated_detailed = [
    (texts[0], '부정', None),
    (texts[1], '긍정', None),
    (texts[2], '보통', None),
    (texts[3], '부정', None),
    (texts[4], '긍정', None)
]

print_statistics(simulated_results, simulated_detailed)


=== 배치 처리 시뮬레이션 ===

=== 감정 분석 통계 ===
긍정: 2개 (40.0%)
부정: 2개 (40.0%)
중립: 1개 (20.0%)

=== 상세 결과 ===
1. 이 영화 정말 재미없어요. 시간 낭비였습니다.... → 부정
2. 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!... → 긍정
3. 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.... → 보통
4. OST는 좋았지만 전개가 늘어졌고 결말이 약했어요.... → 부정
5. 연출이 신선했고 영상미가 뛰어났습니다. 다시 보고 싶어... → 긍정
